In [ ]:
import pathlib

import pandas as pd
from IPython.display import display

import dlsproc.xml
import dlsproc.bundle
import dlsproc.hier
import dlsproc.io
import dlsproc.parse

So that *all* the columns of a `pd.DataFrame` are shown:

In [ ]:
pd.set_option('display.max_columns', None)

The directory containing downloaded data:

In [ ]:
data_directory = pathlib.Path.cwd().parent / 'data'
assert data_directory.exists()
print(data_directory)

/home/manu/dlsproc/data


## Insiders

> Licitaciones publicadas en los perfiles del contratante ubicados en la Plataforma de Contratación del Sector Público, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
insiders_output_file = dlsproc.io.File('insiders')

The directory for *insiders* data

In [ ]:
insiders_directory = data_directory / 'perfiles_plataforma'
assert insiders_directory.exists()
print(insiders_directory)

/home/manu/dlsproc/data/perfiles_plataforma


The data is read from a file if present, or from the given directory if not

In [ ]:
if insiders_output_file.exists():
    insiders_df = insiders_output_file.read()
else:
    # data is read from the given directory, and concatenated into a single `pd.DataFrame`...
    insiders_df = dlsproc.bundle.read_zips_directory(insiders_directory, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    insiders_df = dlsproc.hier.flat_df_to_multiindexed_df(insiders_df)
    
    # ...and saved
    insiders_output_file.write(insiders_df)

In [ ]:
insiders_df.head()

id  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3.atom 0      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 1      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 2      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 3      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 4      https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                  summary  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                    

How many entries and how many fields?

In [ ]:
insiders_df.shape

(78944, 148)

Examples of columns

In [ ]:
insiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyIdentification', 'ID', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'CityName', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'PostalZone', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'AddressLine', 'Line', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'Country', 'IdentificationCode', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PostalAddress', 'Country', 'Name', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'Contact', 'Name', '', '', ''

The first few rows

In [ ]:
insiders_df.head()

id  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3.atom 0      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 1      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 2      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 3      https://contrataciondelestado.es/sindicacion/l...   
                                                                                                 4      https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                  summary  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                    

The types of the first few columns

In [ ]:
insiders_df.dtypes[:4]

id                                            string
summary                                       string
title                                         string
updated                          datetime64[ns, UTC]
dtype: object

Number of levels in the *columns*

In [ ]:
insiders_df.columns.nlevels

12

*updated* is the 3rd column

In [ ]:
insiders_df.columns[3]

('updated', '', '', '', '', '', '', '', '', '', '', '')

In order to extract it *as a time series*

In [ ]:
insiders_df[insiders_df.columns[3]]

zip                                                  file name                                                        entry
licitacionesPerfilesContratanteCompleto3_202201.zip  licitacionesPerfilesContratanteCompleto3.atom                    0       2022-01-31 20:52:58.605000+00:00
                                                                                                                      1       2022-01-31 20:29:04.412000+00:00
                                                                                                                      2       2022-01-31 20:27:36.530000+00:00
                                                                                                                      3       2022-01-31 20:20:41.978000+00:00
                                                                                                                      4       2022-01-31 20:10:35.591000+00:00
                                                                                 

Without specifying all the levels in the index

In [ ]:
insiders_df['updated']

zip                                                  file name                                                        entry
licitacionesPerfilesContratanteCompleto3_202201.zip  licitacionesPerfilesContratanteCompleto3.atom                    0       2022-01-31 20:52:58.605000+00:00
                                                                                                                      1       2022-01-31 20:29:04.412000+00:00
                                                                                                                      2       2022-01-31 20:27:36.530000+00:00
                                                                                                                      3       2022-01-31 20:20:41.978000+00:00
                                                                                                                      4       2022-01-31 20:10:35.591000+00:00
                                                                                 

### Entity

Columns in the `pd.DataFrame` containing "Name" in their names are displayed

In [ ]:
insiders_df[dlsproc.hier.columns_containing(insiders_df, 'Name')].head()

ContractFolderStatus  \
                                                                                                                                  LocatedContractingParty   
                                                                                                                                                    Party   
                                                                                                                                                PartyName   
                                                                                                                                                     Name   
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                                file name                                     entry                                                      
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3.atom 0            Pleno del Ayuntamiento de Alcalá de la Vega   
                                                                                                 1      Concejalia Delegada Area Régimen Interior del ...   
                                                                                                 2                Rectorado de la Universidad de Alicante   
                                                                                                 3                 Alcaldía del Ayuntamiento de Briviesca   
                                                                                                 4                   Alcaldía del Ayuntamiento de Almaraz   

                                                                                                                                        \
                                                                                                                                         
                                                                                                                                         
                                                                                                                         PostalAddress   
                                                                                                                              CityName   
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                                                                                         
                                                                

The two columns below (built from a *path* by means of `dlsproc.xml.assemble_name`) are the most promising ones

In [ ]:
levels_entity_columns = [
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'],
    ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'Contact', 'Name']
]
# entity_columns = [dlsproc.xml.assemble_name(e) for e in levels_entity_columns]
# entity_columns

A `pd.DataFrame` with them is assembled

In [ ]:
names_df = insiders_df.loc[:, [dlsproc.hier.pad_col_levels(insiders_df, c) for c in levels_entity_columns]]
names_df

ContractFolderStatus  \
                                                                                                                                       LocatedContractingParty   
                                                                                                                                                         Party   
                                                                                                                                                     PartyName   
                                                                                                                                                          Name   
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3.atom      0            Pleno del Ayuntamiento de Alcalá de la Vega   
                                                                                                      1      Concejalia Delegada Area Régimen Interior del ...   
                                                                                                      2                Rectorado de la Universidad de Alicante   
                                                                                                      3                 Alcaldía del Ayuntamiento de Briviesca   
                                                                                                      4                   Alcaldía del Ayuntamiento de Almaraz   
..                                                                                                                                                         ...   
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3_202203... 453    Dirección de Compras de la Corporación de Radi...   
                                                                                                      454    Consejero Delegado de la Empresa Municipal de ...   
                                                                                                      455    Jefatura de Intendencia de Asuntos Económicos ...   
                                                                                                      456                Organismo Autónomo Parques Nacionales   
                                                                                                      457    Comité Central de Compras de Navantia S.A., S....   

                                                                                                                                                                
                                                                                        

Are they identical?

In [ ]:
(names_df.iloc[:, 0] == names_df.iloc[:, 1]).all()

True

This column is used from now on

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(insiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

In [ ]:
insiders_df[entity_col]

zip                                                  file name                                                        entry
licitacionesPerfilesContratanteCompleto3_202201.zip  licitacionesPerfilesContratanteCompleto3.atom                    0              Pleno del Ayuntamiento de Alcalá de la Vega
                                                                                                                      1        Concejalia Delegada Area Régimen Interior del ...
                                                                                                                      2                  Rectorado de la Universidad de Alicante
                                                                                                                      3                   Alcaldía del Ayuntamiento de Briviesca
                                                                                                                      4                     Alcaldía del Ayuntamiento de

### Amounts

Columns in the `pd.DataFrame` containing "Amount":

In [ ]:
insiders_df[dlsproc.hier.columns_containing(insiders_df, 'Amount')]

ContractFolderStatus  \
                                                                                                                        ProcurementProject   
                                                                                                                              BudgetAmount   
                                                                                                            EstimatedOverallContractAmount   
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
                                                                                                                                             
zip                                                file name                                          entry                                  
licitacionesPerfilesContratanteCompleto3_202201... licitacionesPerfilesContratanteCompleto3.atom      0                            4919.04   
                                                                                                      1                            5077.92   
                                                                                                      2                          447394.20   
                                                                                                      3                           21600.00   
                                                                                                      4                           66987.95   
..                                                                                                                                     ...   
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3_202203... 453                         28000.00   
                                                                                                      454                          6000.00   
                                                                                                      455                          2727.27   
                                                                                                      456                         86421.00   
                                                                                                      457                        430000.00   

                                                                                                                         \
                                                                                                                          
                                                                                                                          
                                                                                                            TotalAmount   
                                                                                                                          
                                                                                              

Let us focus on this one

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(insiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TotalAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TotalAmount',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows')

The first few values in it

In [ ]:
insiders_df[amount_col].head()

zip                                                  file name                                      entry
licitacionesPerfilesContratanteCompleto3_202201.zip  licitacionesPerfilesContratanteCompleto3.atom  0          1488.00
                                                                                                    1          5077.92
                                                                                                    2        135336.75
                                                                                                    3         22128.00
                                                                                                    4         81055.42
Name: (ContractFolderStatus, ProcurementProject, BudgetAmount, TotalAmount, , , , , , , , ), dtype: float64

A *grouped* `pd.DataFrame` showing the number of contracts and overall amount for each entity

In [ ]:
number_amount_df = insiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , , , , , , )",,
Junta de Gobierno de Val de San Vicente,4.366600e+04,2
13ª Zona de la Guardia Civil - Cantabria,1.584000e+04,2
5ª Zona de la Guardia Civil (Murcia),2.706165e+04,1
8ª Zona de la Guardia Civil Comunidad Autónoma...,0.000000e+00,4
ADIF - Consejo de Administración,4.556022e+09,147
...,...,...
"Área de Gobierno de Urbanismo, Edificación y S...",6.086841e+06,15
Área de Gobierno de Vicealcaldía,8.683304e+07,21
Área de Innovación y Digitalización Urbana del...,2.121195e+05,3


Strangely enough, there are entities with contracts amounting overall to $0$

In [ ]:
number_amount_df.iloc[3]

amount    0.0
number    4.0
Name: 8ª Zona de la Guardia Civil Comunidad Autónoma de Aragón, dtype: float64

In [ ]:
insiders_df.loc[insiders_df[entity_col] == '8ª Zona de la Guardia Civil Comunidad Autónoma de Aragón', :]

id  \
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
zip                                                file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_202203... licitacionesPerfilesContratanteCompleto3_202203... 476    https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 130    https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 62     https://contrataciondelestado.es/sindicacion/l...   
                                                   licitacionesPerfilesContratanteCompleto3_202203... 21     https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                       summary  \
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                                                                                 
                                                                                                         

Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , , , , , , )",,
Dirección General de la Mutualidad de Funciona...,1.066429e+10,5
ADIF Alta Velocidad - Consejo de Administración,5.273203e+09,67
Dirección del Instituto Nacional de Gestión Sa...,4.660165e+09,65
ADIF - Consejo de Administración,4.556022e+09,147
Aena. Consejo de Administración,4.359473e+09,112
...,...,...
Alcaldía del Ayuntamiento de Villaornate y Castro,0.000000e+00,1
Delegación Provincial de la Consejería de Desa...,0.000000e+00,2
Alcaldía del Ayuntamiento de Bugedo,0.000000e+00,2


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents

In [ ]:
documents_df = insiders_df.loc[:, dlsproc.hier.columns_containing(insiders_df, 'DocumentReference')].droplevel(['zip', 'file name'])
documents_df.head()

ContractFolderStatus  \
                               TenderingTerms   
      ProcurementLegislationDocumentReference   
                                           ID   
                                                
                                                
                                                
                                                
                                                
                                                
                                                
                                                
entry                                           
0                                         N/A   
1                                         N/A   
2                                  2014/24/EU   
3                                  2014/24/EU   
4                                         N/A   

                                                      \
                              LegalDocumentReference   
                                                  ID   
                                                       
                                                       
                                                       
                                                       
                                                       
                                                       
                                                       
                                                       
                                                       
entry                                                  
0      PLIEGO CLAUSULAS ADMINISTRATIVAS GESTORIA.pdf   
1                                               <NA>   
2                                               <NA>   
3                              PCAP BAR PISCINAS.pdf   
4                 PCAP CENTRO DIVERSIDAD ALMARAZ.pdf   

                                                          \
                                                           
                                              Attachment   
                                       ExternalReference   
                                                     URI   
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
entry                                                      
0      https://contrataciondelestado.es/wps/wcm/conne...   
1                                                   <NA>   
2                                                   <NA>   
3      https://contrataciondelestado.es/wps/wcm/conne...   
4      https://contrataciondelestado.es/wps/wcm/conne...   

                                     \
                                      
                                      
                                      
                       DocumentHash   
                                      
                                      
                                      
                                      
                                      
                                      
                                      
entry                                 
0      eiM+v83gqxcddXEEKGHlhVeoHGc=   
1                              <NA>   
2                              <NA>   
3      0WZXssCb8B5ajykNl2hd1mSxDvE=   
4      4X7lyZ45lvrENHagw/NsZj3e6pc=   

                                                          \
                              TechnicalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                 

Columns that have been captured

In [ ]:
documents_df.columns.values

array([('ContractFolderStatus', 'TenderingTerms', 'ProcurementLegislationDocumentReference', 'ID', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'Attachment', 'ExternalReference', 'URI', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'LegalDocumentReference', 'Attachment', 'ExternalReference', 'DocumentHash', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'Attachment', 'ExternalReference', 'URI', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'TechnicalDocumentReference', 'Attachment', 'ExternalReference', 'DocumentHash', '', '', '', '', '', '', ''),
       ('ContractFolderStatus', 'AdditionalDocumentReference', 'ID', '', '', '', '', '', '', '', '', ''),
       ('Contr

Let us check one of those ending in a *leaf* named *URI*

In [ ]:
documents_df[
    dlsproc.hier.pad_col_levels(documents_df, ('ContractFolderStatus', 'GeneralDocument', 'GeneralDocumentDocumentReference', 'Attachment', 'ExternalReference', 'URI'))
].iloc[7]

'https://contrataciondelestado.es/wps/wcm/connect/PLACE_es/Site/area/docAccCmpnt?srv=cmpnt&cmpntname=GetDocumentsById&source=library&DocumentIdParam=7219740c-7e47-4607-9e67-3ebda4929656'

## Outsiders

> Licitaciones publicadas en la Plataforma mediante mecanismos de agregación, excluyendo los contratos menores

A file to *cache* the read data

In [ ]:
outsiders_output_file = dlsproc.io.File('outsiders')

The directory for *insiders* data

In [ ]:
outsiders_directory = data_directory / 'agregados'
assert outsiders_directory.exists()
print(outsiders_directory)

/home/manu/dlsproc/data/agregados


In [ ]:
outsiders_zip_file = outsiders_directory / 'PlataformasAgregadasSinMenores_2021.zip'
assert outsiders_zip_file.exists()

The data is read from a file if present, or from the given **zip** if not

In [ ]:
if outsiders_output_file.exists():
    outsiders_df = outsiders_output_file.read()
else:
    # data is read from the given zip, and concatenated into a single `pd.DataFrame`...
    outsiders_df = dlsproc.bundle.read_zip(outsiders_zip_file, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_df)
    
    # ...and saved
    outsiders_output_file.write(outsiders_df)

The data is read from a file if present, or from the given **directory** if not

In [ ]:
%%script false --no-raise-error

if outsiders_output_file.exists():
    outsiders_df = outsiders_output_file.read()
else:
    # data is read from the given directory, and concatenated into a single `pd.DataFrame`...
    outsiders_df = dlsproc.bundle.read_zips_directory(outsiders_directory, concatenate=True)
    
    # ...re-structured with multiindexed columns...
    outsiders_df = dlsproc.hier.flat_df_to_multiindexed_df(outsiders_df)
    
    # ...and saved
    outsiders_output_file.write(outsiders_df)

How many entries and how many fields?

In [ ]:
outsiders_df.shape

(208204, 40)

Examples of columns

In [ ]:
outsiders_df.columns.values[10:20]

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RequiredCommodityClassification', 'ItemClassificationCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'RealizedLocation', 'CountrySubentityCode', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'PlannedPeriod', 'DurationMeasure', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'ProcedureCode', '', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndDate', '', '', ''),
       ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod', 'EndTime', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNotice

The first few rows

In [ ]:
outsiders_df.head()

id  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      https://contrataciondelestado.es/sindicacion/P...   
                                    1      https://contrataciondelestado.es/sindicacion/P...   
                                    2      https://contrataciondelestado.es/sindicacion/P...   
                                    3      https://contrataciondelestado.es/sindicacion/P...   
                                    4      https://contrataciondelestado.es/sindicacion/P...   

                                                                                     summary  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Id licitación: 3064_23/2021; Órgano de Contrat...   
                                    1      Id licitación: 3051_CONT_SERVICIOS/2021/40; Ór...   
                                    2      Id licitación: 3091_21/2021; Órgano de Contrat...   
                                    3      Id licitación: 3069_1/2021; Órgano de Contrata...   
                                    4      Id licitación: 8044_30146-0-21; Órgano de Cont...   

                                                                                       title  \
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                                                               
file name                           entry                                                      
PlataformasAgregadasSinMenores.atom 0      Contrato del Servicio para el desarrollo de la...   
                                    1      REDACCIÓN DEL PLAN ESTRATÉGICO DE ACCESIBILIDA...   
                                    2      Contratación de obra de línea subterránea en A...   
                                    3      Construcción de depósito regulador de 100m3 en...   
                                    4          Renovación de la cubierta de edificio docente   

                                                                   updated  \
                                                                             
                                                                             
                                                                             
                               

Some types

In [ ]:
outsiders_df.dtypes[:20]

id                                                                                                                                        string
summary                                                                                                                                   string
title                                                                                                                                     string
updated                                                                                                                      datetime64[ns, UTC]
ContractFolderStatus  ContractFolderID                                                                                                    string
                      ContractFolderStatusCode                                                                                            string
                      LocatedContractingParty   Party                            PartyName                       Name             

### Entity

Columns in the `pd.DataFrame` containing "Name" in their names are displayed

In [ ]:
outsiders_df[dlsproc.hier.columns_containing(outsiders_df, 'Name')].columns.values

array([('ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'Name', '', '', '', ''),
       ('ContractFolderStatus', 'ValidNoticeInfo', 'AdditionalPublicationStatus', 'PublicationMediaName', '', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'ParentLocatedParty', 'PartyName', 'Name', ''),
       ('ContractFolderStatus', 'TenderResult', 'WinningParty', 'PartyName', 'Name', '', ''),
       ('ContractFolderStatus', 'LocatedContractingParty', 'ParentLocatedParty', 'ParentLocatedParty', 'ParentLocatedParty', 'PartyName', 'Name')],
      dtype=object)

In [ ]:
entity_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'])
entity_col

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '',
 '')

In [ ]:
outsiders_df[entity_col]

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                                  Junta de Gobierno Local
                                                       1                                  Junta de Gobierno Local
                                                       2          ALCALDESA-PRESIDENTA DEL AYUNTAMIENTO DE BUÑUEL
                                                       3                                      CONCEJO DE GALBARRA
                                                       4                                  AYUNTAMIENTO DE BAKAIKU
                                                                                      ...                        
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463                              Directora General de EITB
                                                       464                              Directora General de EITB
                           

### Amounts

Columns in the `pd.DataFrame` containing "Amount":

In [ ]:
outsiders_df[dlsproc.hier.columns_containing(outsiders_df, 'Amount')].columns.values

array([('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'EstimatedOverallContractAmount', '', '', ''),
       ('ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount', '', '', ''),
       ('ContractFolderStatus', 'TenderResult', 'AwardedTenderedProject', 'LegalMonetaryTotal', 'TaxExclusiveAmount', '', '')],
      dtype=object)

In [ ]:
amount_col = dlsproc.hier.pad_col_levels(outsiders_df, ['ContractFolderStatus', 'ProcurementProject', 'BudgetAmount', 'TaxExclusiveAmount'])
amount_col

('ContractFolderStatus',
 'ProcurementProject',
 'BudgetAmount',
 'TaxExclusiveAmount',
 '',
 '',
 '')

In [ ]:
number_amount_df = outsiders_df[[entity_col, amount_col]].groupby(entity_col).agg(amount=(amount_col, 'sum'), number=(amount_col, 'count'))
number_amount_df

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
AGENCIA NAVARRA DE AUTONOMÍA Y DESARROLLO DE L...,1.040800e+07,8
AGENCIA NAVARRA PARA LA AUTONOMIA Y DESARROLLO...,1.796660e+06,2
AJUNTAMENT DE CERVIÀ DE TER,2.198030e+05,4
AJUNTAMENT DE FALSET,2.123535e+06,9
AJUNTAMENT DE SANT JAUME DELS DOMENYS,2.495736e+07,25
...,...,...
presidencia,3.065700e+05,6
Àrea Metropolitana de Barcelona,9.364749e+09,1410
ÁREA DE SALUD I (MURCIA OESTE - H.U. VIRGEN DE...,4.743750e+04,1


Entities with the biggest amounts

In [ ]:
number_amount_df.sort_values('amount', ascending=False)

,amount,number
"(ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, , )",,
Servicio Andaluz de Salud,2.757164e+10,3316
Consejería de Vivienda y Administración Local,2.476717e+10,38
"Canal de Isabel II, S.A.",1.622211e+10,1850
"Empresa Pública de Metro de Madrid, S.A.",1.288923e+10,3323
Ajuntament de Barcelona,1.030976e+10,2380
...,...,...
"ZUGAZTEL, M. P., S. A.",0.000000e+00,0
Directora Gerente Agencia Navarra de Autonomía...,0.000000e+00,0
"ITELAZPI, S.A.",0.000000e+00,0


### Documents

Columns containing *DocumentReference* in the name, allegedly URLs to documents

In [ ]:
# if zip is a "level" of the index...
if 'zip' in outsiders_df.index.names:
    display(outsiders_df.loc[:, dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')].droplevel(['zip', 'file name']))
# if zip is NOT a "level" of the index...
else:
    display(outsiders_df.loc[:, dlsproc.hier.columns_containing(outsiders_df, 'DocumentReference')].droplevel(['file name']))

ContractFolderStatus  \
                             ValidNoticeInfo   
                 AdditionalPublicationStatus   
      AdditionalPublicationDocumentReference   
                                   IssueDate   
                                               
                                               
entry                                          
0                                 2021-12-31   
1                                 2022-01-01   
2                                 2021-12-31   
3                                 2021-12-31   
4                                 2021-12-31   
..                                       ...   
463                               2021-11-26   
464                               2021-11-25   
465                               2021-11-30   
466                               2021-12-29   
467                               2021-12-21   

                                                          \
                                  LegalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4                                                   <NA>   
..                                                   ...   
463    2021.63 PCAP Reforma interior sede IRUÑA EITB.pdf   
464                2021.67 PCAP Suministro de Prensa.pdf   
465                                            PCAP.docx   
466                    3- PCP - LicenciasCV_2022_v11.pdf   
467                                             PCAP.doc   

                                                          \
                                                           
                                              Attachment   
                                       ExternalReference   
                                                     URI   
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4                                                   <NA>   
..                                                   ...   
463    https://www.contratacion.euskadi.eus/ac70cPubl...   
464    https://www.contratacion.euskadi.eus/ac70cPubl...   
465    https://www.contratacion.euskadi.eus/ac70cPubl...   
466    https://www.contratacion.euskadi.eus/ac70cPubl...   
467    https://www.contratacion.euskadi.eus/ac70cPubl...   

                                                          \
                              TechnicalDocumentReference   
                                                      ID   
                                                           
                                                           
                                                           
                                                           
entry                                                      
0                                                   <NA>   
1                                                   <NA>   
2                                                   <NA>   
3                                                   <NA>   
4                                                   <NA>   
..                                               

### Questions

A particular entry

In [ ]:
# outsiders_df.droplevel(['zip', 'file name']).iloc[25000]
outsiders_df.iloc[25000]

id                                                                                                                                                                    https://contrataciondelestado.es/sindicacion/P...
summary                                                                                                                                                               Id licitación: 6012100057;Órgano de Contrataci...
title                                                                                                                                                                  Servicio de gestión de residuos industriales ...
updated                                                                                                                                                                                2021-02-22 12:40:57.479000+00:00
ContractFolderStatus  ContractFolderID                                                                                                  

How many non-null rows provide a value for the entity URL?

In [ ]:
outsiders_df[('ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', '')].notna().sum()

28257

Grouping by exploiting the entity URL

In [ ]:
outsiders_df.groupby(('ContractFolderStatus', 'LocatedContractingParty', 'BuyerProfileURIID', '', '', '', '')).size().sort_values()

(ContractFolderStatus, LocatedContractingParty, BuyerProfileURIID, , , , )
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
https://contractaciopublica.gencat.cat/ecofin_...       1
                                                     ... 
https://www.larioja.org/contratacion-publica/e...     501
http://www.madrid.org/cs/Satellite?cid=1205761...     507
https://contractaciopublica.gencat.cat/ecofin_...     704
https://contractaciopublica.gencat.cat/ecofin_...     712
http://www.madrid.org/cs/Satellite?cid=1204201...    5204
Length: 1203, dtype: int64

A new column with the (internet) domain is added

In [ ]:
outsiders_df['domain'] = dlsproc.parse.domain(outsiders_df)

In [ ]:
outsiders_df['domain']

file name                                              entry
PlataformasAgregadasSinMenores.atom                    0                                <NA>
                                                       1                                <NA>
                                                       2                                <NA>
                                                       3                                <NA>
                                                       4                                <NA>
                                                                            ...             
PlataformasAgregadasSinMenores_20211231_030012_1.atom  463      www.contratacion.euskadi.eus
                                                       464      www.contratacion.euskadi.eus
                                                       465      www.contratacion.euskadi.eus
                                                       466      www.contratacion.euskadi.eus
         

It is used for grouping

In [ ]:
domain_grouped_outsiders_df = outsiders_df.groupby('domain', dropna=False)

The following groups are present, each one with the indicated number of entries

In [ ]:
domain_grouped_outsiders_df.size()

domain
contractaciopublica.gencat.cat    56233
contratacion.euskadi.eus          18944
place-agregacion.carm.es             63
www.asturias.es                    1739
www.bilbao.eus                      989
www.carm.es                           2
www.contratacion.euskadi.eus      17935
www.contratosdegalicia.gal         2400
www.larioja.org                     501
www.madrid.org                    24937
NaN                               84461
dtype: int64

In [ ]:
filled_in_ratio_df = domain_grouped_outsiders_df.apply(lambda x: x.notnull().mean()).drop(dlsproc.hier.pad_col_levels(outsiders_df, ['domain']), axis=1)
filled_in_ratio_df

id summary title updated  \
                                                            
                                                            
                                                            
                                                            
                                                            
                                                            
domain                                                      
contractaciopublica.gencat.cat  1.0     1.0   1.0     1.0   
contratacion.euskadi.eus        1.0     1.0   1.0     1.0   
place-agregacion.carm.es        1.0     1.0   1.0     1.0   
www.asturias.es                 1.0     1.0   1.0     1.0   
www.bilbao.eus                  1.0     1.0   1.0     1.0   
www.carm.es                     1.0     1.0   1.0     1.0   
www.contratacion.euskadi.eus    1.0     1.0   1.0     1.0   
www.contratosdegalicia.gal      1.0     1.0   1.0     1.0   
www.larioja.org                 1.0     1.0   1.0     1.0   
www.madrid.org                  1.0     1.0   1.0     1.0   
NaN                             1.0     1.0   1.0     1.0   

                               ContractFolderStatus                           \
                                   ContractFolderID ContractFolderStatusCode   
                                                                               
                                                                               
                                                                               
                                                                               
                                                                               
domain                                                                         
contractaciopublica.gencat.cat             0.999947                      1.0   
contratacion.euskadi.eus                   1.000000                      1.0   
place-agregacion.carm.es                   1.000000                      1.0   
www.asturias.es                            1.000000                      1.0   
www.bilbao.eus                             1.000000                      1.0   
www.carm.es                                1.000000                      1.0   
www.contratacion.euskadi.eus               1.000000                      1.0   
www.contratosdegalicia.gal                 1.000000                      1.0   
www.larioja.org                            1.000000                      1.0   
www.madrid.org                             1.000000                      1.0   
NaN                                        0.997940                      1.0   

                                                                           \
                               LocatedContractingParty                      
                                                 Party ParentLocatedParty   
                                             PartyName          PartyName   
                                                  Name               Name   
                                                                            
                                                                            
domain                                                                      
contractaciopublica.gencat.cat                     1.0           0.999982   
contratacion.euskadi.eus                           1.0           1.000000   
place-agregacion.carm.es                           1.0           1.000000   
www.asturias.es                                    1.0           0.000000   
www.bilbao.eus                                     1.0           1.000000   
www.carm.es                                        1.0           1.000000   
www.contratacion.euskadi.eus                       1.0           1.000000   
www.contratosdegalicia.gal                         1.0           0.000000   
www.larioja.org                                    1.0           1.000000   
www.madrid.org                             

In [ ]:
# for c in dlsproc.hier.columns_containing(filled_in_ratio, 'LocatedContractingParty'):
#     print(c)

More readable (notice the ordering of the index)

In [ ]:
tidy_filled_in_ratio_df = filled_in_ratio_df.sort_index(axis=1).T
tidy_filled_in_ratio_df

domain                                                                                                                                                        contractaciopublica.gencat.cat  \
ContractFolderStatus ContractFolderID                                                                                                                                               0.999947   
                     ContractFolderStatusCode                                                                                                                                       1.000000   
                     LegalDocumentReference     Attachment                          ExternalReference                      URI                                                      0.833781   
                                                ID                                                                                                                                  0.833781   
                     LocatedContractingParty    BuyerProfileURIID                                                                                                                   0.349314   
                                                ParentLocatedParty                  ParentLocatedParty                     ParentLocatedParty PartyName Name                        0.000000   
                                                                                                                           PartyName          Name                                  0.000000   
                                                                                    PartyName                              Name                                                     0.999982   
                                                Party                               PartyIdentification                    ID                                                       0.000018   
                                                                                    PartyName                              Name                                                     1.000000   
                     ProcurementProject         BudgetAmount                        EstimatedOverallContractAmount                                                                  1.000000   
                                                                                    TaxExclusiveAmount                                                                              1.000000   
                                                Name                                                                                                                                1.000000   
                                                PlannedPeriod                       DurationMeasure                                                                                 0.910889   
                                                                                    EndDate                                                                                         0.088934   
                                                                                    StartDate                                                                                       0.088934   
                                                RealizedLocation                    CountrySubentityCode                                                                            0.999164   
                                                RequiredCommodityClassification     ItemClassificationCode                                                                          0.988992   
                                                TypeCode                                                                                                                            1.000000   
                     TechnicalDocumentReference Attachment                          ExternalReference                      URI                                  

It is saved to disk

In [ ]:
tidy_filled_in_ratio_df.to_excel('outsiders_filled_in.xlsx')

In [ ]:
filled_in_ratio_df.iloc[0]

id                                                                                                                                                                    1.000000
summary                                                                                                                                                               1.000000
title                                                                                                                                                                 1.000000
updated                                                                                                                                                               1.000000
ContractFolderStatus  ContractFolderID                                                                                                                                0.999947
                      ContractFolderStatusCode                                                                               